# Oil Palm Stem Classification using CNN

In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import splitfolders

import tensorflow_addons as tfa

from tensorflow.keras.applications import MobileNetV2, InceptionResNetV2, InceptionV3

import cv2
from skimage.color import rgb2lab
import numpy as np
from typing import Union

import os

DATA_PATH = os.path.join('data', 'selected_dataset_3000')
OUTPUT_PATH = os.path.join('data', 'split_selected_dataset_3000')
TRAIN_PATH = os.path.join(OUTPUT_PATH, 'train')
TEST_PATH = os.path.join(OUTPUT_PATH, 'test')
VAL_PATH = os.path.join(OUTPUT_PATH, 'val')

## Split to train and test

Train will be used for train and validation, while test is used to measure all 6 cnn architecture

In [24]:
splitfolders.ratio(DATA_PATH, output=OUTPUT_PATH, seed=42, ratio=(.8,.1,.1))

## Preprocess and prepare

In [3]:
def enhance(image: Union[str, np.ndarray], display: bool = False) -> np.ndarray:
    """
    Enhance image by using adaptive histogram equalization
    :param display: bool, if true then return converted image in RGB, if not, return as BGR because it needs to continue to another preprocessing
    :param image: [str, np.ndarray], path to image file or image array in numpy array
    :return: np.ndarray, numpy array of enhance image
    """
    if type(image) is str:
        image = cv2.imread(image)
    image = np.uint8(cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX))
    # convert from BGR to YCrCb
    ycrcb = cv2.cvtColor(image, cv2.COLOR_RGB2YCR_CB) # create clahe object
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    # equalize the histogram of the Y channel
    ycrcb[:, :, 0] = clahe.apply(ycrcb[:, :, 0])
    # convert the YCR_CB image back to RGB format
    if display:
        return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2RGB)
    else:
        return cv2.cvtColor(ycrcb, cv2.COLOR_YCR_CB2BGR)

In [4]:
def remove_noise(image: Union[str, np.ndarray], filter: str = 'bilateral', display: bool = False) -> np.ndarray:
    """
    Remove noise from image using bilateral filter
    :param image: [str, np.ndarray], path to image file or image array in numpy array
    :return: np.ndarray, numpy array of image with removed noise
    """
    if type(image) is str:
        image = cv2.imread(image)
    # apply bilateral filter with d = 15, sigmaColor = sigmaSpace = 75.
    if filter == 'bilateral':
        filtered_image = cv2.bilateralFilter(image, 10, 65, 65)
    elif filter == 'median':
        filtered_image = cv2.medianBlur(image, 5)
    else:
        raise ValueError('Unrecognized filter')

    if display:
        return cv2.cvtColor(filtered_image, cv2.COLOR_BGR2RGB)
    return filtered_image

In [20]:
def preprocess(image):
    image = enhance(image) # returned BGR
    image = remove_noise(image, filter='median', display=True) # returned RGB
    image = rgb2lab(image) # returned LAB
    return image

In [21]:
_ = preprocess('data/selected_dataset/infected/DSC03980_4.JPG')
_[:,:,0]

array([[22.50646655, 24.72591622, 25.65447437, ..., 37.24358223,
        35.14120306, 35.14120306],
       [24.72591622, 25.61939422, 29.02538145, ..., 37.24358223,
        35.14120306, 35.14120306],
       [25.94057896, 29.02538145, 30.33127958, ..., 37.32161247,
        35.44240552, 35.14120306],
       ...,
       [77.59049645, 74.0168619 , 67.39515298, ..., 18.95741947,
        18.91942872, 15.2620094 ],
       [74.0168619 , 68.50691067, 61.78662237, ..., 25.34849525,
        25.34849525, 15.2620094 ],
       [74.0168619 , 67.39515298, 60.07142015, ..., 25.75760924,
        25.75760924, 15.2620094 ]])

In [25]:
train_datagen = ImageDataGenerator(
    zoom_range=0.2,
    height_shift_range=0.2,
    width_shift_range=0.2,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess)

train_generator_150 = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(150,150),
    class_mode='binary'
)
val_generator_150 = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(150,150),
    class_mode='binary'
)

train_generator_224 = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(224,224),
    class_mode='binary'
)
val_generator_224 = train_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(224,224),
    class_mode='binary'
)

Found 4340 images belonging to 2 classes.
Found 542 images belonging to 2 classes.
Found 4340 images belonging to 2 classes.
Found 542 images belonging to 2 classes.


## Build model

In [26]:
INPUT_SHAPE = (150, 150, 3)

In [27]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=4,
    verbose=1,
    restore_best_weights=True,
    start_from_epoch=5
)

In [54]:
# TomConv: An Improved CNN Model for Diagnosis of Diseases in Tomato Plant Leaves
# Preeti BaserJatinderkumar R. SainiKetan Kotecha
# https://www.sciencedirect.com/science/article/pii/S1877050923001606
model1 = Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=INPUT_SHAPE),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(256, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model1.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
               loss='binary_crossentropy',
               metrics=['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [28]:
# SWP-LeafNET: A novel multistage approach for plant leaf identification based on deep CNN
# Ali BeikmohammadiKarim FaezAli Motallebi
# https://www.sciencedirect.com/science/article/pii/S0957417422008016
# second model
model2 = Sequential([
    layers.Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(150,150,3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.1),
    layers.Conv2D(160, (3,3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),
    layers.Conv2D(224, (3,3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.3),
    layers.Conv2D(256, (3,3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.AveragePooling2D((2,2)),
    layers.Dropout(0.4),
    layers.Flatten(),
    layers.Dense(512, activation='relu', kernel_regularizer='l2'),
    layers.Dense(256, activation='relu', kernel_regularizer='l2'),
    layers.Dense(64, activation='relu', kernel_regularizer='l2'),
    layers.Dense(1, activation='sigmoid')
])

steps_per_epoch = train_generator_150.samples // train_generator_150.batch_size
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=0.001,
    maximal_learning_rate=0.006,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * steps_per_epoch
)

model2.compile(optimizer=tf.keras.optimizers.Adam(clr),
               loss='binary_crossentropy',
               metrics=['accuracy'])
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_158 (Conv2D)         (None, 150, 150, 64)      1792      
                                                                 
 batch_normalization_158 (B  (None, 150, 150, 64)      256       
 atchNormalization)                                              
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 75, 75, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_159 (Conv2D)         (None, 75, 75, 128)       73856     
                                                                 
 batch_normalization_159 (B  (None, 75, 75, 128)       512       
 atchNormalization)                                              
                                                        

In [19]:
# SWP-LeafNET: A novel multistage approach for plant leaf identification based on deep CNN
# Ali BeikmohammadiKarim FaezAli Motallebi
# https://www.sciencedirect.com/science/article/pii/S0957417422008016
# third model
mobilenet = MobileNetV2(input_shape=(224,224,3),
                        include_top=False,
                        weights='imagenet')

for layer in mobilenet.layers:
    layer.trainable = False

x = layers.Flatten()(mobilenet.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

model3 = tf.keras.models.Model(mobilenet.input, x)

model3.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics='accuracy')

In [20]:
inceptionresnet = InceptionResNetV2(input_shape=INPUT_SHAPE, weights='imagenet', include_top=False)

for layer in inceptionresnet.layers:
    layer.trainable = False

x = layers.Flatten()(inceptionresnet.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

model4 = tf.keras.models.Model(inceptionresnet.input, x)
model4.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])

In [18]:
l_filters = 6
ab_filters = 26
l_filters_3rd = 13
ab_filters_3rd = 51

input = layers.Input(shape=(224,224,3))
l_cnl = layers.Lambda(lambda x: x[:,:,:,0])(input)
l_cnl = layers.Reshape((224,224,1))(l_cnl)
l_cnl = layers.Conv2D(l_filters/2, (3,3), strides=2, activation='relu')(l_cnl)
l_cnl = layers.BatchNormalization()(l_cnl)
l_cnl = layers.Conv2D(l_filters/2, (3,3), activation='relu')(l_cnl)
l_cnl = layers.BatchNormalization()(l_cnl)
l_cnl = layers.Conv2D(l_filters_3rd, (3,3), padding='same', activation='relu')(l_cnl)
l_cnl = layers.BatchNormalization()(l_cnl)
l_cnl = layers.MaxPooling2D(2)(l_cnl)

# # ab_input = layers.Input(shape=(None,224,224,3))
ab_cnl = layers.Lambda(lambda x: x[:,:,:,1:])(input)
ab_cnl = layers.Reshape((224,224,2))(ab_cnl)
ab_cnl = layers.Conv2D(32-ab_filters/2, (3,3), strides=2, activation='relu')(ab_cnl)
ab_cnl = layers.BatchNormalization()(ab_cnl)
ab_cnl = layers.Conv2D(32-ab_filters/2, (3,3), activation='relu')(ab_cnl)
ab_cnl = layers.BatchNormalization()(ab_cnl)
ab_cnl = layers.Conv2D(64-ab_filters_3rd, (3,3), padding='same', activation='relu')(ab_cnl)
ab_cnl = layers.BatchNormalization()(ab_cnl)
ab_cnl = layers.MaxPooling2D(2)(ab_cnl)

x = layers.Concatenate()([l_cnl, ab_cnl])
x = layers.Conv2D(80, (1,1), activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(192, (3,3), activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(2)(x)

# inceptionv3 = InceptionV3(weights='imagenet', include_top=False)
# for inc_layers in inceptionv3.layers:
#     inc_layers.trainable = False
# inception_x6 = inceptionv3.get_layer('mixed6')
# x = inception_x6([x])
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1, activation='sigmoid')(x)

model5 = tf.keras.models.Model(input, x)
model5.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 lambda_14 (Lambda)          (None, 224, 224)             0         ['input_9[0][0]']             
                                                                                                  
 lambda_15 (Lambda)          (None, 224, 224, 2)          0         ['input_9[0][0]']             
                                                                                                  
 reshape_14 (Reshape)        (None, 224, 224, 1)          0         ['lambda_14[0][0]']           
                                                                                            

## Train Model

In [21]:
model1.fit(train_generator_150, validation_data=val_generator_150,
           epochs=105, callbacks=[early_stopping]) # epochs 19 val_accuracy=0.54
model1.save(os.path.join('dumps', 'model1'))

Epoch 1/105
109/109 [==============================] - 134s 1s/step - loss: 0.8932 - accuracy: 0.5147 - val_loss: 0.7443 - val_accuracy: 0.4977
Epoch 2/105
109/109 [==============================] - 100s 918ms/step - loss: 0.7098 - accuracy: 0.5043 - val_loss: 0.7051 - val_accuracy: 0.5000
Epoch 3/105
109/109 [==============================] - 94s 859ms/step - loss: 0.6952 - accuracy: 0.5228 - val_loss: 0.6902 - val_accuracy: 0.5565
Epoch 4/105
109/109 [==============================] - 94s 859ms/step - loss: 0.6942 - accuracy: 0.5170 - val_loss: 0.7325 - val_accuracy: 0.5000
Epoch 5/105
109/109 [==============================] - 95s 869ms/step - loss: 0.6961 - accuracy: 0.5049 - val_loss: 0.6971 - val_accuracy: 0.4977
Epoch 6/105
109/109 [==============================] - 105s 962ms/step - loss: 0.6943 - accuracy: 0.4925 - val_loss: 0.6891 - val_accuracy: 0.5576
Epoch 7/105
109/109 [==============================] - 105s 961ms/step - loss: 0.6935 - accuracy: 0.5115 - val_loss: 0.6903 

INFO:tensorflow:Assets written to: dumps\model1\assets


In [29]:
model2.fit(train_generator_150, validation_data=val_generator_150,
           epochs=105, callbacks=[early_stopping])
model2.save(os.path.join('dumps', 'model2'))

Epoch 1/105
136/136 [==============================] - 392s 3s/step - loss: 4.9338 - accuracy: 0.5661 - val_loss: 0.9453 - val_accuracy: 0.5387
Epoch 2/105
136/136 [==============================] - 333s 2s/step - loss: 0.7982 - accuracy: 0.5597 - val_loss: 0.7291 - val_accuracy: 0.5018
Epoch 3/105
136/136 [==============================] - 302s 2s/step - loss: 0.7408 - accuracy: 0.5359 - val_loss: 0.7040 - val_accuracy: 0.5055
Epoch 4/105
136/136 [==============================] - 293s 2s/step - loss: 0.6913 - accuracy: 0.5988 - val_loss: 0.6757 - val_accuracy: 0.6199
Epoch 5/105
136/136 [==============================] - 291s 2s/step - loss: 0.6880 - accuracy: 0.6071 - val_loss: 0.7085 - val_accuracy: 0.5572
Epoch 6/105
136/136 [==============================] - 289s 2s/step - loss: 0.7042 - accuracy: 0.5968 - val_loss: 0.7095 - val_accuracy: 0.6107
Epoch 7/105
136/136 [==============================] - 293s 2s/step - loss: 0.6930 - accuracy: 0.6065 - val_loss: 0.6798 - val_accuracy:

INFO:tensorflow:Assets written to: dumps\model2\assets


In [23]:
model3.fit(train_generator_224, validation_data=val_generator_224,
           epochs=105, callbacks=[early_stopping])
model3.save(os.path.join('dumps', 'model3'))

Epoch 1/105
109/109 [==============================] - 205s 2s/step - loss: 2.9158 - accuracy: 0.5109 - val_loss: 0.6597 - val_accuracy: 0.6094
Epoch 2/105
109/109 [==============================] - 198s 2s/step - loss: 0.7195 - accuracy: 0.5495 - val_loss: 0.7090 - val_accuracy: 0.5392
Epoch 3/105
109/109 [==============================] - 195s 2s/step - loss: 0.7316 - accuracy: 0.5452 - val_loss: 0.6681 - val_accuracy: 0.5956
Epoch 4/105
109/109 [==============================] - 201s 2s/step - loss: 0.7177 - accuracy: 0.5421 - val_loss: 0.7115 - val_accuracy: 0.5507
Epoch 5/105
109/109 [==============================] - 205s 2s/step - loss: 0.6806 - accuracy: 0.5907 - val_loss: 0.6843 - val_accuracy: 0.5899
Epoch 6/105
109/109 [==============================] - 207s 2s/step - loss: 0.6786 - accuracy: 0.5772 - val_loss: 0.7021 - val_accuracy: 0.5173
Epoch 7/105
109/109 [==============================] - 202s 2s/step - loss: 0.6779 - accuracy: 0.5858 - val_loss: 0.6749 - val_accuracy:

INFO:tensorflow:Assets written to: dumps\model3\assets


In [24]:
model4.fit(train_generator_150, validation_data=val_generator_150,
           epochs=105, callbacks=[early_stopping])
model4.save(os.path.join('dumps', 'model4'))

Epoch 1/105
109/109 [==============================] - 138s 1s/step - loss: 31.9992 - accuracy: 0.4798 - val_loss: 24.5827 - val_accuracy: 0.5046
Epoch 2/105
109/109 [==============================] - 128s 1s/step - loss: 17.6680 - accuracy: 0.4891 - val_loss: 36.4228 - val_accuracy: 0.5012
Epoch 3/105
109/109 [==============================] - 129s 1s/step - loss: 13.3266 - accuracy: 0.5092 - val_loss: 5.1855 - val_accuracy: 0.5127
Epoch 4/105
109/109 [==============================] - 137s 1s/step - loss: 5.7065 - accuracy: 0.4948 - val_loss: 1.9225 - val_accuracy: 0.5081
Epoch 5/105
109/109 [==============================] - 133s 1s/step - loss: 3.3969 - accuracy: 0.5127 - val_loss: 6.6688 - val_accuracy: 0.5046
Epoch 6/105
109/109 [==============================] - 133s 1s/step - loss: 2.7693 - accuracy: 0.5207 - val_loss: 1.4295 - val_accuracy: 0.4839
Epoch 7/105
109/109 [==============================] - 132s 1s/step - loss: 2.1191 - accuracy: 0.5037 - val_loss: 0.8661 - val_accu

INFO:tensorflow:Assets written to: dumps\model4\assets
